### Intro
- Rasa core and Rasa NLU combined is called RASA STACK
- We will build a weather reporting chatbot
    - this will do Entity Extraction and Intent classification task
    
### Setup
1. pip install -r requirements.txt
2. language model ( en ) 
    - lanuage model is used to parse incoming text messages and extract necessary information
3. Rasa NLU trainer
    - this makes generating training data lot easier
    - this is a UI, and js based application, so we will need npm and nodejs
    - download node js with npm and add to path
    - then run
        - npm i -g rasa-nlu-trainer
    - this will install rasa-nlu trainer

In [1]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

In [2]:
# 2. install language model 

import sys
python = sys.executable

# this will download english spacy model
    # it will install it and reference it to abbreviation en
!{python} -m spacy download en

    100% |████████████████████████████████| 37.4MB 92.1MB/s ta 0:00:01   2% |▊                               | 860kB 5.1MB/s eta 0:00:08    4% |█▎                              | 1.5MB 30.3MB/s eta 0:00:02    4% |█▌                              | 1.7MB 33.4MB/s eta 0:00:02    4% |█▋                              | 1.9MB 2.6MB/s eta 0:00:14/s eta 0:00:01    17% |█████▌                          | 6.4MB 6.9MB/s eta 0:00:05    18% |██████                          | 7.0MB 11.3MB/s eta 0:00:03    21% |██████▉                         | 8.0MB 1.4MB/s eta 0:00:21    23% |███████▌                        | 8.8MB 1.7MB/s eta 0:00:17    29% |█████████▌                      | 11.1MB 5.1MB/s eta 0:00:06    33% |██████████▊                     | 12.5MB 8.9MB/s eta 0:00:03    39% |████████████▌                   | 14.6MB 6.7MB/s eta 0:00:04    39% |████████████▋                   | 14.8MB 2.1MB/s eta 0:00:11    43% |██████████████                  | 16.4MB 3.5MB/s eta 0:00:06    44% |██████████████▏     

### Training 
- we have to teach a chatbot how to understand human unstructured language so that bot will understand what we are saying
    - So we will train NLU model which will take unstructured text messages and will return structured data in the form of intents and entities which our bot will understand
- Intent
    - what the message is about
- Entity
    - informations like location names, dates etc
    - this helps chatbot to understand what specifically we are talking about and asking
- Entity Extraction and Intent classification are ML problems, so we need train data to train the models

##### Rasa NLU train data
- train data should contain example messages which we would like our chatbot to learn from
    - the corresponding intents and what entities included in each sentence and where in a sentence they can be found
    
##### creating data
- mkdir data
- cd data
- echo 'nlu_data' > data.json
- there are two different ways of how we can create training examples for NLU models
    - one way is to directly write them into this data file
    
{
  "rasa_nlu_data":{
    "common_examples":[
    {
       "text":"Hello",
       "intent":"greet",
       "entities":[]
    },
    {
       "text":"goodbye",
       "intent":"goodbye",
       "entities":[]
    }
      ]
  }
}

    - save this file
    - another way :
        - in the data folder, launch rasa-nlu-trainer
        - here we add new example
        - add text
            - What's the weather in Berlin at the moment?
        - now highlight berlin and add as entity,
            - give entity name as location
        - Now if we open data.json, we can see entites populated 
            - also we have start and end to show where entity is present

#### Amount of training data
- now we have three examples of training 
- we need more examples for each of these intents
    - examples should be different and diverse
- now add data from data.json from github to our data.json
    - here we have some more examples of greeting, goodbye and asking for weather
- reload nlu_trainer
- now we will have close to 40 examples in total, which is also less

#### Before training
- we need to create a configuration file
- go out of data folder,
    - echo 'config' > config_spacy.json
- config file is imp as it provides some params to be used
    - 1. pipeline : this will specify what featurizers,feature extractors are going to be used to crunch text messages and extract neccesary info in RASA NLU
        - Rasa NLU has two main pipelines pre-built
            - a. MIDI based
            - b. Sklearn based
    - 2. path : dir where we will keep model after training
    - 3. data : data file location
    
- config_spacy.json
{
  "pipeline":"spacy_sklearn",
  "path":"./models/nlu",
  "data":"./data/data.json"
}

#### model creation

In [2]:
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer
from rasa_nlu.model import Metadata, Interpreter

In [3]:
def train_nlu(data, configs, model_dir):
    training_data = load_data(data)
    # we will have to provide a config in trainer,
        # this we wil do using RasaNLUConfig method
    trainer = Trainer(config.load(configs))
    trainer.train(training_data)
    # model dir is where our model is saved
    model_directory = trainer.persist( model_dir, fixed_model_name= "weathernlu")

In [4]:
train_nlu("./data/data.json", "config_spacy.json", "./models/nlu")

INFO:rasa_nlu.training_data.loading:Training data format of ./data/data.json is rasa_nlu
INFO:rasa_nlu.training_data.training_data:Training data stats: 
	- intent examples: 29 (3 distinct intents)
	- Found intents: 'greet', 'inform', 'goodbye'
	- entity examples: 13 (1 distinct entities)
	- found entities: 'location'

INFO:rasa_nlu.utils.spacy_utils:Trying to load spacy model with name 'en'
INFO:rasa_nlu.components:Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.
INFO:rasa_nlu.model:Starting to train component nlp_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component tokenizer_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_featurizer_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_entity_featurizer_regex
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component ner_cr

Fitting 2 folds for each of 6 candidates, totalling 12 fits


- to check if model is persisted look for models folder
- Metadata and Intepreter class is required to load the model and get ready to use it

In [5]:
def run_nlu():
    # now we load our model
    intepreter = Interpreter.load("./models/nlu/default/weathernlu")
    pprint(intepreter.parse(u"I am planning my holiday to Lithuania. I wonder what is the weather out there."))

In [6]:
run_nlu()

INFO:rasa_nlu.components:Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.


{
  "intent": {
    "name": "inform",
    "confidence": 0.8509291192465037
  },
  "entities": [
    {
      "start": 28,
      "end": 37,
      "value": "lithuania",
      "entity": "location",
      "confidence": 0.919550976223837,
      "extractor": "ner_crf"
    }
  ],
  "intent_ranking": [
    {
      "name": "inform",
      "confidence": 0.8509291192465037
    },
    {
      "name": "goodbye",
      "confidence": 0.07978649851713138
    },
    {
      "name": "greet",
      "confidence": 0.06928438223636452
    }
  ],
  "text": "I am planning my holiday to Lithuania. I wonder what is the weather out there."
}


- RASA NLU tells what intent is the text
- also it tells the confidence score for all other intents that we have

### Dialogue Management model
- Dialogue management model will predict what action or resonse the chatbot should make based on the state of the conversation
    - actions can be simple API calls or text responses or retriving data from DB
- Q. Why to we need ML for that ?
    - Now if we ask for weather without giving our location, we want our chatbot to ask for what location we are based
        - In practice these type of conversations are hard coded in form of flow charts.
        - code wise we can image this as bunch of if else statements
        - This means a developer has to create a bunch of most possible happy paths from starting the conversation to the end goal.
        - now with every intent and entity we add, the flowchart becomes complex and difficult to monitor all possible paths that user can make to get the answers they want
    - In Rasa, we have a ML model which we can train and it will make a prediction of what the bot should do next based on
        - a. the context
        - b. and the state of the conversation
        - as a result, the conversational flow is way more natural and we have a better user experience
        
### Building dialogue management model
- create a domain file for our chatbot
- weather_domain.yml
    - this will be a yaml file
- domain describes an enivronment
- domain consists of 5 key parts
    - 1. list of slots 
        - slots are like placeholders that would help chatbot to keep track of context of the conversation
            - ex. we are asking weather in specific location
                - so chatbot should keep track of the location that we are asking. And we do not remind the chatbot of what location we were speaking initially. So it can keep track of the location in further 
                - also chatbot is going to make an api call to get the weather information
        - so we will create a slot called location, also we will need to tell what data type this slot is going to have
            - datatype is imp as diff data types of the slots are going to have effect on how dialogue management model is going to make predictions
            - for some datatypes value will be imp in predictions
            - for some, whether the slot is populated or not is going to have impact on the prediction made
        - in our case, location type will be text
        - here we will have only one slot for our example
                
    - 2. intents
        - these are same intents that we had in NLU model
            - we had three
                 - greet
                 - goodbye
                 - inform
    - 3. entities
        - list of entities that chatbot should be aware of and ready of get from user
            - we had only one
                - location 
            - here we have an entity called location as well as slot. So NLU model will extract the location name as an entity and will set this value as a slot.
                - thats how this value is going to be saved and kept throughout the conversation
    - 4. list of templates
        - are like text responses that chatbot should send back to the user once specific actions are being predicted
        - so we will initialize an action which should be executed when my 
        - corresponding to an actiong we will write the text message that we want to reply
            - ex. utter_greet:
                - 'hello ! how can I help ?'
            - utter_goodbye:
                - 'Talk to you later'
                - 'Bye Bye :(' 
        - we provided more diversity so we added on more possible answer, our chatbot will randomize a little to which answer it will use
            - utter_ask_location:
                - 'In what location?'
    - 5. list of actions
        - actions that my chatbot should be ready to execute when they are predicted
        - we already have 3 actions create in template
            - utter_greet
            - utter_goodbye
            - utter_ask_location
        - note : for returning weather data, we will have a custom action and use python code
    - all these 5 are imp as they will be used in dialogue management model to make predictions by RASA Core Dialogue Management model
        - RCDM model will make prediction on what actions should me executed next on the slots that are currently populated based on 
            - a. intents and entities returned by Rasa NLU model ie. what a user spoke about
            - b. what actions were performed previously ie. what is the state of the conversation at the moment
          
### Custom Actions  

In [34]:
# from apixu.client import ApixuClient
# client = ApixuClient("3564bf1fbe6d44d0b4c93136190906")
# client.forecast(q="India")

{'current': {'cloud': 0,
  'condition': {'code': 1030,
   'icon': '//cdn.apixu.com/weather/64x64/night/143.png',
   'text': 'Mist'},
  'feelslike_c': 38.8,
  'feelslike_f': 101.9,
  'gust_kph': 18.0,
  'gust_mph': 11.2,
  'humidity': 31,
  'is_day': 0,
  'last_updated': '2019-06-09 21:45',
  'last_updated_epoch': 1560096909,
  'precip_in': 0.0,
  'precip_mm': 0.0,
  'pressure_in': 29.9,
  'pressure_mb': 998.0,
  'temp_c': 37.0,
  'temp_f': 98.6,
  'uv': 0.0,
  'vis_km': 3.5,
  'vis_miles': 2.0,
  'wind_degree': 0,
  'wind_dir': 'N',
  'wind_kph': 0.0,
  'wind_mph': 0.0},
 'forecast': {'forecastday': [{'astro': {'moonrise': '11:15 AM',
     'moonset': 'No moonset',
     'sunrise': '05:23 AM',
     'sunset': '07:18 PM'},
    'date': '2019-06-09',
    'date_epoch': 1560038400,
    'day': {'avghumidity': 9.0,
     'avgtemp_c': 46.5,
     'avgtemp_f': 115.6,
     'avgvis_km': 17.5,
     'avgvis_miles': 10.0,
     'condition': {'code': 1000,
      'icon': '//cdn.apixu.com/weather/64x64/day/1

In [35]:
# future enables new language features which may not compatible with
    # current interpreter
# So that code will work with older version of python

from __future__ import absolute_import, division, unicode_literals

from rasa_core.actions.action import Action
from rasa_core.events import SlotSet

# 
class ActionWeather(Action):
    
    # create name of the action
    def name(self):
        return "action_weather"
    
    # here all the action will happen
    # apixu
    def run(self, dispatcher, tracker, domain):
        from apixu.client import ApixuClient
        api_key = "3564bf1fbe6d44d0b4c93136190906"
        # authentication
        client = ApixuClient(api_key)
        # remember we have a slot which keeps location info
            # through out the conversation
        # from tracker get value of a particular slot    
        loc = tracker.get_slot("location")
        
        # response is going to be a dictionary
            # having lot of details
        current = client.forecast(q=loc)
        
        # now we will parse the response
        country = current['location']['country']
        city = current['location']['name']
        condition = current['current']['condition']['text']
        temp_c = current['current']['temp_c']
        humidity = current['current']['humidity']
        wind_mph = current['current']['wind_mph']
        
        # now we will create response message
        response = """
            It is currently {} in {} at the moment. The temperature is {} degrees,
             The Humidity is {}% and the wind speed is {} mph.""".format(condition,
                                                                         city, 
                                                                         temp_c,
                                                                         humidity,
                                                                         wind_mph);
        # dispatcher will send out the response
        dispatcher.utter_message(response)
        
        # lastly we will return current slot value
        return [SlotSet('location',loc)]

- now we have to include this to our domain
    - under actions
        - actions.ActionWeather
    - note : here Actionweather class is in file actions.py
    - in jupyter add __main__.ActionWeather

In [36]:
domain_yml_data = """
slots:
  location:
    type: text


intents:
 - greet
 - goodbye
 - inform


entities:
 - location

templates:
  utter_greet:
    - 'Hello! How can I help?'
  utter_goodbye:
    - 'Talk to you later.'
    - 'Bye bye :('
  utter_ask_location:
    - 'In what location?'


actions:
 - utter_greet
 - utter_goodbye
 - utter_ask_location
 - __main__.ActionWeather
""" 
 
%store domain_yml_data > weather_domain.yml

Writing 'domain_yml_data' (str) to file 'weather_domain.yml'.


### Data
- dialogue management model is trained on actual conversations that users have with the bot
    - only thing is that these conversations have to be converted into a story format
    - story is and actual conversation b/w a user and a chatbot where user inputs are converted into correspoding intents and entities while response of chatbots are expressed as actions which the bot would execute at that specific stage
- example of how a real conversation can be written as a story
<img src="rasa1.JPG">

- but before a question arises is how to get this data in the first place
    - One way is to use Rasa core NLU feature called Online Training
        - this will not only help with generating data, but also train a dialogue management model in real time

#### Generating Stories
- go to data folder and create stories.md
    - this will be markdown extension
- we will create some stateless stories
    - stateless stories are conversations where we have one user input and one user response
- * intent : conversation that will start with intent

In [37]:
stories = """

## story 01
* greet
    - utter_greet

## story 02
* goodbye
    - utter_goodbye
    
## story 03
* inform
    - utter_ask_location

## story 04
* inform
    - action_weather
"""

%store stories > stories.md

Writing 'stories' (str) to file 'stories.md'.


- we can start online training session

### training
- Agent : it is going to train the model
- KerasPolicy/ Memoization policy : models to be used to train

In [38]:

from __future__ import absolute_import
from __future__ import division
from __future__ import unicode_literals

import logging

from rasa_core.agent import Agent
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy

logging.basicConfig(level='INFO')

training_data_file = 'stories.md'

# where to save model once trained
model_path = './models/dialogue'

import os

In [39]:
agent = Agent('weather_domain.yml', policies = [MemoizationPolicy(), KerasPolicy()])

agent.train(
training_data_file,
augmentation_factor = 50,
#max_history = 2,
epochs = 500,
batch_size = 10,
validation_split = 0.2)

agent.persist(model_path)

Processed Story Blocks: 100%|██████████| 4/4 [00:00<00:00, 170.25it/s, # trackers=20]
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by MaxHistoryTrackerFeaturizer)...
Processed trackers: 100%|██████████| 152/152 [00:01<00:00, 90.63it/s, # actions=169]
INFO:rasa_core.featurizers:Created 169 action examples.
Processed actions: 169it [00:00, 649.41it/s, # examples=127]
INFO:rasa_core.policies.memoization:Memorized 127 unique action examples.
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by MaxHistoryTrackerFeaturizer)...
Processed trackers: 100%|██████████| 152/152 [00:01<00:00, 88.50it/s, # actions=169]
INFO:rasa_core.featurizers:Created 169 action examples.
INFO:rasa_core.policies.keras_policy:Fitting model with 169 total samples and a validation split of 0.2


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_8 (Masking)          (None, 5, 11)             0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 32)                5632      
_________________________________________________________________
dense_8 (Dense)              (None, 6)                 198       
_________________________________________________________________
activation_8 (Activation)    (None, 6)                 0         
Total params: 5,830
Trainable params: 5,830
Non-trainable params: 0
_________________________________________________________________
Train on 135 samples, validate on 34 samples
Epoch 1/500
135/135 [==============================] - 1s - loss: 1.7579 - acc: 0.2889 - val_loss: 1.7025 - val_acc: 0.5294
Epoch 2/500
135/135 [==============================] - 0s - loss: 1.6668 - acc: 0.4741 - val_loss: 1.62

135/135 [==============================] - 0s - loss: 0.3326 - acc: 0.8444 - val_loss: 0.2350 - val_acc: 0.8235
Epoch 60/500
135/135 [==============================] - 0s - loss: 0.2942 - acc: 0.8370 - val_loss: 0.2313 - val_acc: 0.8235
Epoch 61/500
135/135 [==============================] - 0s - loss: 0.2813 - acc: 0.8370 - val_loss: 0.2355 - val_acc: 0.8529
Epoch 62/500
135/135 [==============================] - 0s - loss: 0.3199 - acc: 0.8296 - val_loss: 0.2204 - val_acc: 0.8235
Epoch 63/500
135/135 [==============================] - 0s - loss: 0.2884 - acc: 0.8741 - val_loss: 0.2154 - val_acc: 0.8235
Epoch 64/500
135/135 [==============================] - 0s - loss: 0.2837 - acc: 0.8519 - val_loss: 0.2111 - val_acc: 0.8235
Epoch 65/500
135/135 [==============================] - 0s - loss: 0.2838 - acc: 0.8593 - val_loss: 0.2072 - val_acc: 0.8235
Epoch 66/500
135/135 [==============================] - 0s - loss: 0.2734 - acc: 0.8519 - val_loss: 0.2058 - val_acc: 0.8529
Epoch 67/500


135/135 [==============================] - 0s - loss: 0.2449 - acc: 0.8889 - val_loss: 0.1822 - val_acc: 0.8235
Epoch 125/500
135/135 [==============================] - 0s - loss: 0.2111 - acc: 0.8741 - val_loss: 0.1823 - val_acc: 0.8235
Epoch 126/500
135/135 [==============================] - 0s - loss: 0.2133 - acc: 0.8815 - val_loss: 0.1821 - val_acc: 0.8235
Epoch 127/500
135/135 [==============================] - 0s - loss: 0.2165 - acc: 0.8741 - val_loss: 0.1819 - val_acc: 0.8235
Epoch 128/500
135/135 [==============================] - 0s - loss: 0.2083 - acc: 0.8815 - val_loss: 0.1866 - val_acc: 0.8235
Epoch 129/500
135/135 [==============================] - 0s - loss: 0.2105 - acc: 0.8815 - val_loss: 0.1867 - val_acc: 0.8235
Epoch 130/500
135/135 [==============================] - 0s - loss: 0.2234 - acc: 0.8519 - val_loss: 0.1897 - val_acc: 0.8235
Epoch 131/500
135/135 [==============================] - 0s - loss: 0.2235 - acc: 0.8741 - val_loss: 0.1905 - val_acc: 0.8235
Epoch 

135/135 [==============================] - 0s - loss: 0.1944 - acc: 0.8815 - val_loss: 0.1970 - val_acc: 0.8235
Epoch 190/500
135/135 [==============================] - 0s - loss: 0.2118 - acc: 0.8741 - val_loss: 0.1989 - val_acc: 0.8235
Epoch 191/500
135/135 [==============================] - 0s - loss: 0.2029 - acc: 0.8815 - val_loss: 0.2016 - val_acc: 0.8235
Epoch 192/500
135/135 [==============================] - 0s - loss: 0.2021 - acc: 0.8815 - val_loss: 0.1999 - val_acc: 0.8235
Epoch 193/500
135/135 [==============================] - 0s - loss: 0.2021 - acc: 0.8593 - val_loss: 0.1963 - val_acc: 0.8235
Epoch 194/500
135/135 [==============================] - 0s - loss: 0.2045 - acc: 0.8593 - val_loss: 0.1985 - val_acc: 0.8235
Epoch 195/500
135/135 [==============================] - 0s - loss: 0.1953 - acc: 0.8741 - val_loss: 0.1988 - val_acc: 0.8235
Epoch 196/500
135/135 [==============================] - 0s - loss: 0.1888 - acc: 0.8889 - val_loss: 0.2033 - val_acc: 0.8235
Epoch 

...

135/135 [==============================] - 0s - loss: 0.1996 - acc: 0.8519 - val_loss: 0.2060 - val_acc: 0.8235
Epoch 320/500
135/135 [==============================] - 0s - loss: 0.1808 - acc: 0.8889 - val_loss: 0.2145 - val_acc: 0.8235
Epoch 321/500
135/135 [==============================] - 0s - loss: 0.1951 - acc: 0.8741 - val_loss: 0.2173 - val_acc: 0.8235
Epoch 322/500
135/135 [==============================] - 0s - loss: 0.2130 - acc: 0.8519 - val_loss: 0.2127 - val_acc: 0.8235
Epoch 323/500
135/135 [==============================] - 0s - loss: 0.1997 - acc: 0.8370 - val_loss: 0.2037 - val_acc: 0.8235
Epoch 324/500
135/135 [==============================] - 0s - loss: 0.1890 - acc: 0.8963 - val_loss: 0.2073 - val_acc: 0.8235
Epoch 325/500
135/135 [==============================] - 0s - loss: 0.1864 - acc: 0.8741 - val_loss: 0.2103 - val_acc: 0.8235
Epoch 326/500
135/135 [==============================] - 0s - loss: 0.1975 - acc: 0.8741 - val_loss: 0.2121 - val_acc: 0.8235
Epoch 

135/135 [==============================] - 0s - loss: 0.1902 - acc: 0.8815 - val_loss: 0.2100 - val_acc: 0.8235
Epoch 385/500
135/135 [==============================] - 0s - loss: 0.2030 - acc: 0.8741 - val_loss: 0.2131 - val_acc: 0.8235
Epoch 386/500
135/135 [==============================] - 0s - loss: 0.1843 - acc: 0.8889 - val_loss: 0.2209 - val_acc: 0.8235
Epoch 387/500
135/135 [==============================] - 0s - loss: 0.1915 - acc: 0.8667 - val_loss: 0.2169 - val_acc: 0.8235
Epoch 388/500
135/135 [==============================] - 0s - loss: 0.1853 - acc: 0.8741 - val_loss: 0.2166 - val_acc: 0.8235
Epoch 389/500
135/135 [==============================] - 0s - loss: 0.1915 - acc: 0.8889 - val_loss: 0.2126 - val_acc: 0.8235
Epoch 390/500
135/135 [==============================] - 0s - loss: 0.1860 - acc: 0.8593 - val_loss: 0.2144 - val_acc: 0.8235
Epoch 391/500
135/135 [==============================] - 0s - loss: 0.1934 - acc: 0.8667 - val_loss: 0.2158 - val_acc: 0.8235
Epoch 

135/135 [==============================] - 0s - loss: 0.1838 - acc: 0.8963 - val_loss: 0.2313 - val_acc: 0.8235
Epoch 449/500
135/135 [==============================] - 0s - loss: 0.1918 - acc: 0.8519 - val_loss: 0.2240 - val_acc: 0.8235
Epoch 450/500
135/135 [==============================] - 0s - loss: 0.1897 - acc: 0.8667 - val_loss: 0.2245 - val_acc: 0.8235
Epoch 451/500
135/135 [==============================] - 0s - loss: 0.1875 - acc: 0.8815 - val_loss: 0.2286 - val_acc: 0.8235
Epoch 452/500
135/135 [==============================] - 0s - loss: 0.1952 - acc: 0.8519 - val_loss: 0.2221 - val_acc: 0.8235
Epoch 453/500
135/135 [==============================] - 0s - loss: 0.1915 - acc: 0.8963 - val_loss: 0.2243 - val_acc: 0.8235
Epoch 454/500
135/135 [==============================] - 0s - loss: 0.1864 - acc: 0.8593 - val_loss: 0.2229 - val_acc: 0.8235
Epoch 455/500
135/135 [==============================] - 0s - loss: 0.1876 - acc: 0.8815 - val_loss: 0.2230 - val_acc: 0.8235
Epoch 

INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Model directory ./models/dialogue exists and contains old model files. All files will be overwritten.


INFO:rasa_core.agent:Persisted model to '/home/vivek/Desktop/jupyter files/RASA-Chatbot/building chatbot-jpboost yt/models/dialogue'


### Train online
- all the files save in above step will be required to create a dialogue management model which we will launch in online training session
    - online training session will improve our chatbot
- ConsoleInputChannel : online trainig session is an actual conversation with a chatbot. we will send messages using console
- RasaNLUModel : we will load our model 

##### whats special
- here, before replying with some text msg or action, chatbot will tell us what it thinks it should do at that specific state of the conversation and it will ask for feedback
    - if we see that the chatbot is about to make some mistakes we will be able to tell this and it will learn from the feedback and it will improve over time

In [28]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import logging

from rasa_core.agent import Agent
from rasa_core.channels.console import ConsoleInputChannel
from rasa_core.interpreter import RegexInterpreter
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.interpreter import RasaNLUInterpreter

logger = logging.getLogger(__name__)

In [29]:
def run_weather_online(input_channel, interpreter,
                          domain_file="weather_domain.yml",
                          training_data_file='data/stories.md'):
    agent = Agent(domain_file,
                  policies=[MemoizationPolicy(), KerasPolicy()],
                  interpreter=interpreter)

    agent.train_online(training_data_file,
                       input_channel=input_channel,
                      # max_history=2,
                       batch_size=50,
                       epochs=200,
                       max_training_samples=300)
    return agent

In [40]:
logging.basicConfig(level="INFO")
nlu_interpreter = RasaNLUInterpreter('./models/nlu/default/weathernlu')
run_weather_online(ConsoleInputChannel(), nlu_interpreter)

INFO:rasa_nlu.components:Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.
Processed Story Blocks: 100%|██████████| 3/3 [00:00<00:00, 170.64it/s, # trackers=20]
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by MaxHistoryTrackerFeaturizer)...
Processed trackers: 100%|██████████| 84/84 [00:00<00:00, 87.83it/s, # actions=79]
INFO:rasa_core.featurizers:Created 79 action examples.
Processed actions: 79it [00:00, 607.12it/s, # examples=79]
INFO:rasa_core.policies.memoization:Memorized 79 unique action examples.
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by MaxHistoryTrackerFeaturizer)...
Processed trackers: 100%|██████████| 84/84 [00:00<00:00, 90.61it/s, # actions=79]
INFO:rasa_core.featurizers:Created 79 action examples.
INFO:rasa_core.policies.keras_policy:Fitting model with 79 total samples and a validation split of 0.0


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_9 (Masking)          (None, 5, 11)             0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 32)                5632      
_________________________________________________________________
dense_9 (Dense)              (None, 6)                 198       
_________________________________________________________________
activation_9 (Activation)    (None, 6)                 0         
Total params: 5,830
Trainable params: 5,830
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
79/79 [==============================] - 0s - loss: 1.8086 - acc: 0.1899     
Epoch 2/200
79/79 [==============================] - 0s - loss: 1.7494 - acc: 0.3671     
Epoch 3/200
79/79 [==============================] - 0s - loss: 1.7143 - acc: 0.3671     
Ep

79/79 [==============================] - 0s - loss: 0.9573 - acc: 0.5570     
Epoch 82/200
79/79 [==============================] - 0s - loss: 0.9547 - acc: 0.5696     
Epoch 83/200
79/79 [==============================] - 0s - loss: 0.9499 - acc: 0.5696     
Epoch 84/200
79/79 [==============================] - 0s - loss: 0.9266 - acc: 0.5570     
Epoch 85/200
79/79 [==============================] - 0s - loss: 0.9165 - acc: 0.5443     
Epoch 86/200
79/79 [==============================] - 0s - loss: 0.9114 - acc: 0.5949     
Epoch 87/200
79/79 [==============================] - 0s - loss: 0.9168 - acc: 0.6456     
Epoch 88/200
79/79 [==============================] - 0s - loss: 0.9330 - acc: 0.5949     
Epoch 89/200
79/79 [==============================] - 0s - loss: 0.8773 - acc: 0.6076     
Epoch 90/200
79/79 [==============================] - 0s - loss: 0.8700 - acc: 0.5949     
Epoch 91/200
79/79 [==============================] - 0s - loss: 0.8594 - acc: 0.6329     
Epoch 92/200

79/79 [==============================] - 0s - loss: 0.2355 - acc: 0.9873     
Epoch 170/200
79/79 [==============================] - 0s - loss: 0.2200 - acc: 1.0000     
Epoch 171/200
79/79 [==============================] - 0s - loss: 0.2555 - acc: 0.9873     
Epoch 172/200
79/79 [==============================] - 0s - loss: 0.1915 - acc: 1.0000     
Epoch 173/200
79/79 [==============================] - 0s - loss: 0.2242 - acc: 1.0000     
Epoch 174/200
79/79 [==============================] - 0s - loss: 0.2416 - acc: 0.9873     
Epoch 175/200
79/79 [==============================] - 0s - loss: 0.2126 - acc: 0.9620     
Epoch 176/200
79/79 [==============================] - 0s - loss: 0.1864 - acc: 0.9873     
Epoch 177/200
79/79 [==============================] - 0s - loss: 0.2025 - acc: 1.0000     
Epoch 178/200
79/79 [==============================] - 0s - loss: 0.2339 - acc: 0.9873     
Epoch 179/200
79/79 [==============================] - 0s - loss: 0.2437 - acc: 0.9367     
Ep

INFO:rasa_core.policies.keras_policy:Done fitting keras policy model


...

thanks! The bot will now [action_weather]
 -----------


Processed trackers: 100%|██████████| 5/5 [00:00<00:00, 94.48it/s, # actions=28]
INFO:rasa_core.featurizers:Created 28 action examples.
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by MaxHistoryTrackerFeaturizer)...
Processed trackers: 100%|██████████| 5/5 [00:00<00:00, 104.19it/s, # actions=22]
INFO:rasa_core.featurizers:Created 22 action examples.
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by MaxHistoryTrackerFeaturizer)...
Processed trackers: 100%|██████████| 5/5 [00:00<00:00, 76.54it/s, # actions=28]
INFO:rasa_core.featurizers:Created 28 action examples.
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by MaxHistoryTrackerFeaturizer)...
Processed trackers: 100%|██████████| 5/5 [00:00<00:00, 76.00it/s, # actions=28]
INFO:rasa_core.featurizers:Created 28 action examples.
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by

Processed trackers: 100%|██████████| 5/5 [00:00<00:00, 73.94it/s, # actions=36]
INFO:rasa_core.featurizers:Created 36 action examples.
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by MaxHistoryTrackerFeaturizer)...
Processed trackers: 100%|██████████| 5/5 [00:00<00:00, 65.16it/s, # actions=32]
INFO:rasa_core.featurizers:Created 32 action examples.
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by MaxHistoryTrackerFeaturizer)...
Processed trackers: 100%|██████████| 5/5 [00:00<00:00, 72.62it/s, # actions=30]
INFO:rasa_core.featurizers:Created 30 action examples.
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by MaxHistoryTrackerFeaturizer)...
Processed trackers: 100%|██████████| 5/5 [00:00<00:00, 89.18it/s, # actions=26]
INFO:rasa_core.featurizers:Created 26 action examples.
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by 


            It is currently Mist in New Delhi at the moment. The temperature is 37.0 degrees,
             The Humidity is 31% and the wind speed is 0.0 mph.
------
Chat history:

	bot did:	utter_ask_location

	bot did:	action_listen

	user did:	inform

	with location:	india

	bot did:	action_weather

we currently have slots: location: india

------
The bot wants to [action_listen]. Is this correct?

	1.	Yes.
	2.	No, the action is wrong.
	0.	Export current conversations as stories and quit
1
Next user input:
Goodbye
------
Chat history:

	bot did:	action_listen

	user did:	inform

	with location:	india

	bot did:	action_weather

	bot did:	action_listen

	user said:	Goodbye

		 whose intent is:	goodbye

we currently have slots: location: india

------
The bot wants to [utter_goodbye] due to the intent. Is this correct?

	1.	Yes
	2.	No, intent is right but the action is wrong
	3.	The intent is wrong
	0.	Export current conversations as stories and quit
1
Bye bye :(
------
Chat history:



INFO:rasa_core.policies.online_trainer:Stories got exported to '/home/vivek/Desktop/jupyter files/RASA-Chatbot/building chatbot-jpboost yt/stories1.md'.


- we said hello,
    - the chat bot did understand we are greeting as NLU model classified this message as greet
    - It didnt find any entites so it didnt populate any slots
    - bot tells us that it wants to execute utter_greet 
    - we have to select the correct option 
- chatbot responded with hello how can i help ?
    - now the bot wants to execute action_listen 
        - action_listen is an inbuilt rasa core action, which means it will wait for next input from a user
- next user input : we will ask What's the weather?
    - bot identified intent as inform, found no entity as there are none
    - bot wants to execute action ask location which is correct
- next user input : In India
    - now it got the intent as inform, and it recognized our entity India
    - now our bot wants to utter_ask_location again which does not makes sense as we already told what location 
        - so our chatbot is about to make a mistake
        - so we can provide our feedback
        - Since intent was correct but action incorrect we will select number 2
- now bot will ask which action to select in this case
    - we will select action_weather
    - it will reply with weather response
- we say Goodbye
    - bot will say goodbye

### result
- all the changes we provided to the chat bot are going to be saved automatically and the model is being improved as we are doing this training session 
- Additionally we can get the conversation as training data sample
    - choose export current conversation as stories and quit 
    - we have to provide a file where we want to save the generated data
    - now we saved this as stories1.md
    - lets open stories1.md
- We can see generated story, so that entire conversation that we had with our chatbot
    - it is in rasa core stories format, so we can simply take our conversation and add this to our training sample and use it in our model.
    
### updating training data
- our current training data contains few stateless stories
    - We will copy this new generated story to our training data ie. stories.md ( removing older data ) 
- Of course one story is not enough to train the model
- add examples from stories.md from github and retrain the model
    - ie. few generated and few manually written stories
    - more stories , more diverse the stories are better the chatbot will perform
- Now we will use this to train the final dialogue management model and then we will integrate with slack

### final dialogue management model

In [41]:
def train_dialogue(domain_file = "weather_domain.yml", 
                  model_path = "./models/dialogue",
                  training_data_file = 'stories.md'):
    agent = Agent(domain_file, policies= [MemoizationPolicy(), KerasPolicy()])
    
    agent.train( training_data_file, max_visual_history=2, epochs=300,
                batch_size=50, validation_split = 0.2, augmentation_factor = 50 )
    
    agent.persist(model_path)
    
    return agent

# load the agent and test it out in console
    # Now we are going use console, so we will check if the server
        # is constantly listening to incoming msgs
def run_weather_bot(server_forever=True):
    # load our NLU model
    interpreter = RasaNLUInterpreter('./models/nlu/default/weathernlu')
    
    agent = Agent.load("./models/dialogue", interpreter= interpreter)
    
    # if server is ready
    if server_forever:
        # initialize console channel
        agent.handle_channel(ConsoleInputChannel())
        
    return agent

In [42]:
train_dialogue()
run_weather_bot()

Processed Story Blocks: 100%|██████████| 11/11 [00:00<00:00, 216.55it/s, # trackers=5]
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by MaxHistoryTrackerFeaturizer)...
Processed trackers: 100%|██████████| 36/36 [00:00<00:00, 45.34it/s, # actions=39]
INFO:rasa_core.featurizers:Created 39 action examples.
Processed actions: 39it [00:00, 464.66it/s, # examples=39]
INFO:rasa_core.policies.memoization:Memorized 39 unique action examples.
INFO:rasa_core.featurizers:Creating states and action examples from collected trackers (by MaxHistoryTrackerFeaturizer)...
Processed trackers: 100%|██████████| 36/36 [00:00<00:00, 41.15it/s, # actions=39]
INFO:rasa_core.featurizers:Created 39 action examples.
INFO:rasa_core.policies.keras_policy:Fitting model with 39 total samples and a validation split of 0.2


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_10 (Masking)         (None, 5, 11)             0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 32)                5632      
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 198       
_________________________________________________________________
activation_10 (Activation)   (None, 6)                 0         
Total params: 5,830
Trainable params: 5,830
Non-trainable params: 0
_________________________________________________________________
Train on 31 samples, validate on 8 samples
Epoch 1/300
31/31 [==============================] - 0s - loss: 1.7554 - acc: 0.3226 - val_loss: 1.7564 - val_acc: 0.3750
Epoch 2/300
31/31 [==============================] - 0s - loss: 1.7334 - acc: 0.3871 - val_loss: 1.7340 - v

31/31 [==============================] - 0s - loss: 1.2063 - acc: 0.5484 - val_loss: 1.3370 - val_acc: 0.5000
Epoch 61/300
31/31 [==============================] - 0s - loss: 1.1855 - acc: 0.5484 - val_loss: 1.3343 - val_acc: 0.5000
Epoch 62/300
31/31 [==============================] - 0s - loss: 1.2171 - acc: 0.5484 - val_loss: 1.3308 - val_acc: 0.5000
Epoch 63/300
31/31 [==============================] - 0s - loss: 1.1818 - acc: 0.5484 - val_loss: 1.3258 - val_acc: 0.5000
Epoch 64/300
31/31 [==============================] - 0s - loss: 1.1455 - acc: 0.5484 - val_loss: 1.3232 - val_acc: 0.5000
Epoch 65/300
31/31 [==============================] - 0s - loss: 1.1842 - acc: 0.5484 - val_loss: 1.3182 - val_acc: 0.5000
Epoch 66/300
31/31 [==============================] - 0s - loss: 1.1541 - acc: 0.5484 - val_loss: 1.3143 - val_acc: 0.5000
Epoch 67/300
31/31 [==============================] - 0s - loss: 1.1893 - acc: 0.5484 - val_loss: 1.3097 - val_acc: 0.5000
Epoch 68/300
31/31 [=========

31/31 [==============================] - 0s - loss: 0.8290 - acc: 0.6452 - val_loss: 1.0451 - val_acc: 0.5000
Epoch 127/300
31/31 [==============================] - 0s - loss: 0.8205 - acc: 0.6452 - val_loss: 1.0399 - val_acc: 0.5000
Epoch 128/300
31/31 [==============================] - 0s - loss: 0.7906 - acc: 0.6774 - val_loss: 1.0321 - val_acc: 0.5000
Epoch 129/300
31/31 [==============================] - 0s - loss: 0.7774 - acc: 0.7097 - val_loss: 1.0263 - val_acc: 0.5000
Epoch 130/300
31/31 [==============================] - 0s - loss: 0.7916 - acc: 0.6774 - val_loss: 1.0213 - val_acc: 0.5000
Epoch 131/300
31/31 [==============================] - 0s - loss: 0.7812 - acc: 0.6452 - val_loss: 1.0166 - val_acc: 0.5000
Epoch 132/300
31/31 [==============================] - 0s - loss: 0.8096 - acc: 0.6452 - val_loss: 1.0107 - val_acc: 0.5000
Epoch 133/300
31/31 [==============================] - 0s - loss: 0.7596 - acc: 0.6129 - val_loss: 1.0048 - val_acc: 0.5000
Epoch 134/300
31/31 [=

31/31 [==============================] - 0s - loss: 0.3882 - acc: 1.0000 - val_loss: 0.7004 - val_acc: 0.8750
Epoch 193/300
31/31 [==============================] - 0s - loss: 0.4689 - acc: 0.8710 - val_loss: 0.7031 - val_acc: 0.8750
Epoch 194/300
31/31 [==============================] - 0s - loss: 0.4409 - acc: 0.9032 - val_loss: 0.6927 - val_acc: 0.8750
Epoch 195/300
31/31 [==============================] - 0s - loss: 0.4128 - acc: 0.9032 - val_loss: 0.6908 - val_acc: 0.8750
Epoch 196/300
31/31 [==============================] - 0s - loss: 0.4679 - acc: 0.8387 - val_loss: 0.6890 - val_acc: 0.8750
Epoch 197/300
31/31 [==============================] - 0s - loss: 0.4528 - acc: 0.9677 - val_loss: 0.6894 - val_acc: 0.8750
Epoch 198/300
31/31 [==============================] - 0s - loss: 0.4734 - acc: 0.8710 - val_loss: 0.6917 - val_acc: 0.8750
Epoch 199/300
31/31 [==============================] - 0s - loss: 0.3948 - acc: 1.0000 - val_loss: 0.6829 - val_acc: 0.8750
Epoch 200/300
31/31 [=

31/31 [==============================] - 0s - loss: 0.2419 - acc: 1.0000 - val_loss: 0.4780 - val_acc: 0.8750
Epoch 259/300
31/31 [==============================] - 0s - loss: 0.2360 - acc: 0.9677 - val_loss: 0.4727 - val_acc: 0.8750
Epoch 260/300
31/31 [==============================] - 0s - loss: 0.2579 - acc: 0.9677 - val_loss: 0.4719 - val_acc: 0.8750
Epoch 261/300
31/31 [==============================] - 0s - loss: 0.2576 - acc: 0.9677 - val_loss: 0.4720 - val_acc: 0.8750
Epoch 262/300
31/31 [==============================] - 0s - loss: 0.2363 - acc: 1.0000 - val_loss: 0.4693 - val_acc: 0.8750
Epoch 263/300
31/31 [==============================] - 0s - loss: 0.2534 - acc: 0.9355 - val_loss: 0.4680 - val_acc: 0.8750
Epoch 264/300
31/31 [==============================] - 0s - loss: 0.2217 - acc: 1.0000 - val_loss: 0.4671 - val_acc: 0.8750
Epoch 265/300
31/31 [==============================] - 0s - loss: 0.2169 - acc: 1.0000 - val_loss: 0.4617 - val_acc: 0.8750
Epoch 266/300
31/31 [=

INFO:rasa_core.policies.keras_policy:Done fitting keras policy model
INFO:rasa_core.agent:Model directory ./models/dialogue exists and contains old model files. All files will be overwritten.


INFO:rasa_core.agent:Persisted model to '/home/vivek/Desktop/jupyter files/RASA-Chatbot/building chatbot-jpboost yt/models/dialogue'
INFO:rasa_nlu.components:Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.


Bot loaded. Type a message and press enter: 
hello
Hello! How can I help?
tell me the weather
In what location?
In Russia

            It is currently Sunny in Moscow at the moment. The temperature is 26.0 degrees,
             The Humidity is 48% and the wind speed is 13.6 mph.
goodbye
Talk to you later.


KeyboardInterrupt: 

- so we have a chatbot that is working.
- now we can stop right here, or build some UI.

### Integration with slack
- We need to build a slack app bcz only slack app can send interactive msgs to slack
    - interactive messages are msgs with buttons, pictures etc
- open https://api.slack.com/
    - login and go to this page
        - create a workspace called botspace
    - click start building, create an app named Alfred in botspace
    - Now we will get add features and functionality page
- Click bots 
    - click add a bot user
    - always show my bot as online : when we create a bot
        - it will be like a user, so do we want to show it online all the time or not
        - select on and click add bot user
        
- now if we go down, we will see Display Information, ie. we can see a little info about our bot
    - ie. we can set description and picture
- now we need to install this app in our workplace
    - below all new features we can see install this app to your workplace
    
#### adding python script
- now we need a python script to integrate the chatbot that we created locally to this slack app.
- rasa_slack_connector.py
    - our connector will have 2 main parts
        - input channel and output channel
        - input channel will take msgs from slack and pass them through RASA core and RASA nlu
        - output channel will take msgs returned by rasa core and send them back to slack 
    - so we will create two classes, one for input channel and other for output channel
    
- we will have our main part which is a server
    - since my app will communicate with my chat bot using HTTP request we will use flask
    

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import logging

from builtins import str
from flask import Blueprintm request, jsonify

from rasa_core.channels.channel import UserMessage, OutputChannel
from rasa_core.channels.rest import HttpInputComponent

logger = logging.getLogger(__name__)

class Slackbot(OutputChannel):
    
    

class SlackInput(HttpInputComponent):
    
    def __init__(self, slack_dev_token, slack_verification_token, slack_client, debug_mode):
        self.slack_dev_token = slack_dev_token
        self.debug_mode = debug_mode
        self.slack_client = slack_client
        self.slack_verification_token = slack_verification_token
        
    def blueprint(self, on_new_message):